In [1]:
from random import seed
from random import random
from math import exp
import numpy as np
import pandas as pd
import textwrap
import timeit
import binascii

In [2]:
def addZeros(strr, n): 
    for i in range(n): 
        strr = "0" + strr 
    return strr 

def getXOR(a, b): 
    aLen = len(a) 
    bLen = len(b) 


    if (aLen > bLen): 
        b = addZeros(b, aLen - bLen) 
    elif (bLen > aLen): 
        a = addZeros(a, bLen - aLen) 
 
    lenn = max(aLen, bLen); 

    res = "" 
    for i in range(lenn): 
        if (a[i] == b[i]): 
            res += "0"
        else: 
            res += "1"
    return res 

In [3]:
def text_to_bits(text, encoding='utf-8', errors='surrogatepass'):
    bits = bin(int(binascii.hexlify(text.encode(encoding, errors)), 16))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))

def text_from_bits(bits, encoding='utf-8', errors='surrogatepass'):
    n = int(bits, 2)
    return int2bytes(n).decode(encoding, errors)

def int2bytes(i):
    hex_string = '%x' % i
    n = len(hex_string)
    return binascii.unhexlify(hex_string.zfill(n + (n & 1)))

In [4]:
#initializing the network
def initialize_network(parameters):
    network = list()
    for j in range(1,len(parameters)):
        layer = [{'weights':[random() for i in range(parameters[j-1] + 1)]} for i in range(parameters[j])]
        network.append(layer)
    return network

In [5]:
# Calculate neuron activation for an input
def activate(weights, inputs):
    #print(inputs)
    activation = weights[-1]
    for i in range(len(weights)-1):
        #print(i)
        #print(type(weights[i]))
        #print(type(inputs[i]))
        activation += weights[i] * inputs[i]
    return activation

In [6]:
# Transfer neuron activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

In [7]:
# Forward propagate input to a network output
def forward_propagate(network, row, getEncryption):
    count = 0
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        if count == 1 and getEncryption == 1:
            print("io")
            return new_inputs
        inputs = new_inputs
        count += 1
  
    return inputs

In [8]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [9]:
# Update network weights with error
def update_weights(network, row, l_rate):
    
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    count = 1
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            count += 1
            outputs = forward_propagate(network, row, 0)
            expected = row[-1]
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        if count % 100 == 0:
            print('> epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
        count += 1

In [10]:
#batch training
# Train a network for a fixed number of epochs
def train_network_batch(network, train, l_rate, n_epoch, n_outputs):
    count = 1
    k = 0
    batch = 10
    for i in range(batch):
        print('> Batch = %d' % (i+1))
        for epoch in range(n_epoch):
            sum_error = 0
            for row in train[k:k+10]:
                count += 1
                outputs = forward_propagate(network, row, 0)
                expected = row[-1]
                sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
                backward_propagate_error(network, expected)
                update_weights(network, row, l_rate)
            if count % 100 == 0:
                print('    epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
            count += 1
        k += 10

In [11]:
import string
a = string.printable
ascii_list = list()
for i in a:
    ascii_list.append(text_to_bits(i))
encrypted = list()
c = 1
for i in ascii_list:
    t = np.random.randint(2, size= 8)
    "".join(map(str, t))
    t = "".join([str(a) for a in t])
    if t in encrypted or t in ascii_list:
        flag = 1
        while flag:
            t = np.random.randint(2, size= 8)
            "".join(map(str, t))
            t = "".join([str(a) for a in t])
            if t in encrypted or t in ascii_list:
                flag = 1
            else:
                encrypted.append(t)
                flag = 0
    else:
        encrypted.append(t)

In [12]:
#testing new random dataset
flag = 1
for i in encrypted:
    if i in ascii_list:
        flag = 0
        print("dataset compromised")
        break
l = len(encrypted)
c = len(list( dict.fromkeys(encrypted)))
if flag == 1 and l == c:
    print("dataset clean")

dataset clean


In [13]:
count = 0
dataset = list()
for i in ascii_list:
    k = list(i)
    k = list(map(int,k))
    j = list(encrypted[count])
    j = list(map(int,j))
    count += 1
    k.append(j)
    dataset.append(k)

# Testing data creation for 7 bit data

In [593]:
a = string.printable
ascii_list = list()
for i in a:
    j = text_to_bits(i)
    ascii_list.append(j[1:])
encrypted = list()
c = 1
for i in ascii_list:
    t = np.random.randint(2, size= 7)
    "".join(map(str, t))
    t = "".join([str(a) for a in t])
    if t in encrypted or t == i:
        flag = 1
        while flag:
            t = np.random.randint(2, size= 7)
            "".join(map(str, t))
            t = "".join([str(a) for a in t])
            if t in encrypted or t == i:
                flag = 1
            else:
                encrypted.append(t)
                flag = 0
    else:
        encrypted.append(t)

In [599]:
#testing new random dataset
flag = 1
l = len(encrypted)
c = len(list( dict.fromkeys(encrypted)))
if flag == 1 and l == c:
    print("dataset clean")

dataset clean


In [600]:
count = 0
dataset = list()
for i in ascii_list:
    k = list(i)
    k = list(map(int,k))
    j = list(encrypted[count])
    j = list(map(int,j))
    count += 1
    k.append(j)
    dataset.append(k)

# Test data creation ended

In [608]:
seed(1)
network = initialize_network([7,16,16,7])


In [609]:
n_inputs = len(dataset[0]) - 1
n_outputs = len(dataset[0][-1])
start = timeit.default_timer()
train_network(network, dataset, 0.5, 10000, n_outputs)
stop = timeit.default_timer()
print("time = {}".format(stop - start))

> epoch=99, lrate=0.500, error=207.751
> epoch=199, lrate=0.500, error=163.068
> epoch=299, lrate=0.500, error=106.988
> epoch=399, lrate=0.500, error=57.451
> epoch=499, lrate=0.500, error=37.579
> epoch=599, lrate=0.500, error=30.401
> epoch=699, lrate=0.500, error=24.469
> epoch=799, lrate=0.500, error=20.049
> epoch=899, lrate=0.500, error=18.344
> epoch=999, lrate=0.500, error=17.572
> epoch=1099, lrate=0.500, error=17.225
> epoch=1199, lrate=0.500, error=16.976
> epoch=1299, lrate=0.500, error=17.204
> epoch=1399, lrate=0.500, error=17.257
> epoch=1499, lrate=0.500, error=19.684
> epoch=1599, lrate=0.500, error=16.644
> epoch=1699, lrate=0.500, error=16.534
> epoch=1799, lrate=0.500, error=16.445
> epoch=1899, lrate=0.500, error=16.406
> epoch=1999, lrate=0.500, error=16.365
> epoch=2099, lrate=0.500, error=16.337
> epoch=2199, lrate=0.500, error=16.313
> epoch=2299, lrate=0.500, error=16.292
> epoch=2399, lrate=0.500, error=16.274
> epoch=2499, lrate=0.500, error=16.258
> epoch=

In [21]:
# Make a prediction with a network
def predict(network, row, encr):
    outputs = forward_propagate(new_network, row, encr)
    k = [round(i) for i in outputs]
    return k

In [681]:
k = list()
for i in range(100):
    k.append(predict(network, dataset[i], 0))
    k[i].append(dataset[i][:-1])

In [682]:
k

[[1, 1, 0, 1, 1, 1, 0, 1, [0, 0, 1, 1, 0, 0, 0, 0]],
 [1, 1, 0, 0, 1, 1, 1, 0, [0, 0, 1, 1, 0, 0, 0, 1]],
 [1, 1, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 0, 1, 0]],
 [1, 0, 1, 1, 1, 1, 1, 0, [0, 0, 1, 1, 0, 0, 1, 1]],
 [1, 0, 1, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 0, 0]],
 [0, 0, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 0, 1]],
 [1, 0, 0, 1, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 1, 0]],
 [1, 0, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 1, 1]],
 [1, 1, 0, 0, 1, 0, 1, 0, [0, 0, 1, 1, 1, 0, 0, 0]],
 [0, 0, 0, 1, 1, 1, 0, 0, [0, 0, 1, 1, 1, 0, 0, 1]],
 [1, 0, 1, 0, 1, 0, 1, 0, [0, 1, 1, 0, 0, 0, 0, 1]],
 [1, 0, 0, 1, 0, 1, 1, 1, [0, 1, 1, 0, 0, 0, 1, 0]],
 [1, 0, 0, 0, 1, 1, 0, 1, [0, 1, 1, 0, 0, 0, 1, 1]],
 [1, 0, 1, 1, 0, 1, 0, 0, [0, 1, 1, 0, 0, 1, 0, 0]],
 [1, 0, 0, 1, 1, 0, 0, 1, [0, 1, 1, 0, 0, 1, 0, 1]],
 [1, 0, 0, 0, 0, 0, 1, 1, [0, 1, 1, 0, 0, 1, 1, 0]],
 [1, 0, 0, 1, 1, 1, 0, 0, [0, 1, 1, 0, 0, 1, 1, 1]],
 [1, 1, 0, 1, 1, 1, 0, 0, [0, 1, 1, 0, 1, 0, 0, 0]],
 [1, 0, 1, 0, 0, 0, 0, 1, [0, 1, 1, 0, 1, 0, 0

In [611]:

c = list()
for i in k:
    s = ""
    i = list(map(str,i))
    c.append(s.join(i))

In [612]:
len(list( dict.fromkeys(c)))

87

In [649]:
k

[[0, 0, 1, 1, 0, 0, 1, [0, 1, 1, 0, 0, 0, 0]],
 [0, 0, 0, 0, 0, 0, 0, [0, 1, 1, 0, 0, 0, 1]],
 [1, 1, 0, 0, 0, 0, 0, [0, 1, 1, 0, 0, 1, 0]],
 [1, 0, 1, 1, 1, 1, 0, [0, 1, 1, 0, 0, 1, 1]],
 [0, 0, 1, 0, 0, 1, 0, [0, 1, 1, 0, 1, 0, 0]],
 [0, 0, 1, 0, 1, 0, 1, [0, 1, 1, 0, 1, 0, 1]],
 [1, 0, 0, 0, 0, 0, 1, [0, 1, 1, 0, 1, 1, 0]],
 [1, 0, 0, 1, 1, 1, 1, [0, 1, 1, 0, 1, 1, 1]],
 [0, 1, 1, 1, 1, 0, 1, [0, 1, 1, 1, 0, 0, 0]],
 [1, 0, 1, 1, 1, 0, 0, [0, 1, 1, 1, 0, 0, 1]],
 [0, 1, 0, 0, 1, 0, 1, [1, 1, 0, 0, 0, 0, 1]],
 [0, 1, 0, 1, 1, 1, 0, [1, 1, 0, 0, 0, 1, 0]],
 [0, 1, 0, 1, 1, 1, 1, [1, 1, 0, 0, 0, 1, 1]],
 [1, 0, 1, 0, 1, 1, 0, [1, 1, 0, 0, 1, 0, 0]],
 [1, 1, 1, 0, 0, 0, 1, [1, 1, 0, 0, 1, 0, 1]],
 [1, 0, 0, 0, 1, 1, 0, [1, 1, 0, 0, 1, 1, 0]],
 [1, 0, 1, 1, 0, 0, 0, [1, 1, 0, 0, 1, 1, 1]],
 [0, 1, 0, 1, 1, 0, 1, [1, 1, 0, 1, 0, 0, 0]],
 [0, 1, 1, 1, 0, 1, 1, [1, 1, 0, 1, 0, 0, 1]],
 [0, 1, 1, 1, 0, 1, 1, [1, 1, 0, 1, 0, 1, 0]],
 [0, 1, 0, 1, 1, 0, 0, [1, 1, 0, 1, 0, 1, 1]],
 [1, 0, 1, 0,

In [292]:
t = list()
t.append(predict(network,dataset[0],0))
for i in range(1,100):
    if predict(network,dataset[i],0) in t:
        print(i)
        index = i
        break
    t.append(predict(network,dataset[i],0))

75


In [320]:
t = np.random.randint(2, size= 8)
"".join(map(str, t))
p = "".join([str(a) for a in t])
if p in c:
    flag = 1
    while flag:
        t = np.random.randint(2, size= 8)
        "".join(map(str, t))
        p = "".join([str(a) for a in t])
        if p in c:
            flag = 1
        else:
            flag = 0

In [321]:
c = list()
for i in k:
    s = ""
    i = list(map(str,i))
    c.append(s.join(i))

In [322]:
len(list( dict.fromkeys(c)))

100

In [323]:
count = 0
dataset = list()
for i in ascii_list:
    p = list(i)
    p = list(map(int,p))
    j = list(k[count])
    j = list(map(int,j))
    count += 1
    p.append(j)
    dataset.append(p)

In [328]:
type(network)

list

In [331]:
import json
my_list = ['a', 'b', 'c']
my_json_string = json.dumps(my_list)

In [335]:
my_network_json = json.dumps(network)

In [15]:
# -*- coding: utf-8 -*-
import json

# Make it work for Python 2+3 and with Unicode
import io
try:
    to_unicode = unicode
except NameError:
    to_unicode = str

In [511]:
# Write JSON file
with io.open('../network/network.json', 'w', encoding='utf8') as outfile:
    str_ = json.dumps(network,
                      indent=4, sort_keys=True,
                      separators=(',', ': '), ensure_ascii=False)
    outfile.write(to_unicode(str_))

In [39]:
# Read JSON file
with open('../network/network.json') as data_file:
    new_network = json.load(data_file)

In [17]:
data = list()
f=open("../asciiTextData/initialTestDocument.txt","r")
line=f.readline()
while line!='':
    data.append(line)
    print(line)
    line=f.readline()    

Hello good afternoon ladies and gentlemen.

This is the official testing document for encryptor & decryptor efficiency.

The following message is just a garbage and is only checked for efficiency

A

G

BhjUi

$342@/.dj

jSjjd


In [51]:
new_data =list()
for i in ascii_list:
    new_data.append(list(map(int,list(i))))

In [53]:
for i in new_data:
    i.append(0)

In [60]:
en = list()
for i in range(len(new_data)):
    en.append(predict(new_network,new_data[i],0))

In [61]:
for i in range(len(new_data)):
    en[i].append(new_data[i][:-1])

In [63]:
# Write JSON file
with io.open('../datasets/languageDecryptor.json', 'w', encoding='utf8') as outfile:
    str_ = json.dumps(en,
                      indent=4, sort_keys=True,
                      separators=(',', ': '), ensure_ascii=False)
    outfile.write(to_unicode(str_))

In [18]:
bit_data = list()
for i in data:
    bit_data.append(text_to_bits(i))
flag = 1
for i in range(len(bit_data)):
    if ((len(bit_data[i]))%8) != 0:
        print("data corrupted")
        flag = 0
        break

if flag:
    print("data conversion preserved")

data conversion preserved


In [19]:
input_data = list()
for i in bit_data:
    l = len(i)
    k = int(l/8)
    c = 0
    for j in range(k):
        s = i[c:c+8]
        input_data.append(s)
        c += 8
        
c = 0
for i in bit_data:
    l = len(i)
    c += int(l/8)

if c == len(input_data):
    print("data input is ready")

input_data_final = list()
for i in input_data:
    a = list(i)
    b = list(i)
    a = list(map(int,a))
    b = list(map(int,b))
    a.append(b)
    input_data_final.append(a)

data input is ready


In [22]:
encrypted_data = list()
for i in input_data_final:
    encrypted_data.append(predict(new_network,i,0))

In [23]:
encrypted_data_final = list()
for i in encrypted_data:
    s = ""
    i = list(map(str,i))
    encrypted_data_final.append(s.join(i))

In [24]:
s = ""
for i in encrypted_data_final:
    s += i

In [621]:
s = list(s)

In [624]:
# Write JSON file
with io.open('../asciiTextData/encrypted.json', 'w', encoding='utf8') as outfile:
    str_ = json.dumps(s,
                      indent=4, sort_keys=True,
                      separators=(',', ': '), ensure_ascii=False)
    outfile.write(to_unicode(str_))

In [633]:
file1 = open("../asciiTextData/encrypted.txt","w")

In [634]:
file1.write(s) 
file1.close()

In [635]:
file1 = open("../asciiTextData/encrypted.txt","r+")  
  
print("Output of Read function is ")
s = file1.read()

Output of Read function is 


In [637]:
import zlib, base64
file = open('../asciiTextData/encrypted.txt','r')
text = file.read()
file.close()
print("Uncompressed Text: \n"+text)
# encoding the text
code =  base64.b64encode(zlib.compress(text.encode('utf-8'),9))
code = code.decode('utf-8')
f=open('../asciiTextData/encrypted.txt','w')
f.write(code)
f.close()
print("Compressed text: \n"+code)
# decode the encoded text
decoded_txt = zlib.decompress(base64.b64decode(code))
print(decoded_txt)

Uncompressed Text: 
00000001100110011111010111110101110100011110011110011100110100011101000110110100111001111010101010000011111001101001100110010101000110001101000111010001000110001110011111110101101010101011010010100001100110010001011011100111101010100001100010110100111001111001110010011001000110001110011011110101100110011101010110011001000110000001001110110111101111011101110010100001000101101110011110100001000101101110011111100110110111001001100111100111110100011000001110000011101000011000110110100001101010101111010111100111111001101001100100010110111001101010000100011000100111001110011110110100110100011000110111111101110101011001100100011000111001101110011110000011110100011001010111100111100110010001100010001101100101011001100010111111111001101101000110010101111001111100101111100111101101001001100110001101100101011001100010111111111001101101000110010101111001111001100110000011100000111010000110001101101000011001100100011000100011011001100000010011101101111011110111011100100110011110

In [638]:
file = open('../asciiTextData/encrypted.txt','r')
text = file.read()
file.close()

In [639]:
text

'eNqtVIENwzAIe8n8/9yqFYOdhKrS1qnJlhFjMAB8nwi+14PQ/Vrvc1m/xnmee560HT8AYcskX9TdHa/PySr5FG6C3R4aE/nXwqPwNp75Ve6LX7Rf5gf2m/YAUetSLsQPYOB3PC//zi9UpfIdskusDBaSt9bX/ZzzZ7yjg1T+vGxaUrEpX5JfRgVJSNWH6eQ8Mj7DR7mW/B3xomw9DgjWL/iC806fOb539bXUx5rPqa/RffWo80HHZOZdiYnPVP/oCRGH+lvnyda37Rell82fkc/zXJj6cptPXh8S11EfOQd0Hm5z6VU//Ku+OkjIfI22lWikt6H6uf/GQ/dNRawNJoNoqXPBFz40okC+3/c+XlZMGg=='

In [640]:
decoded_txt = zlib.decompress(base64.b64decode(text))

In [643]:
output = decoded_txt.decode()

In [644]:
output

'000000011001100111110101111101011101000111100111100111001101000111010001101101001110011110101010100000111110011010011001100101010001100011010001110100010001100011100111111101011010101010110100101000011001100100010110111001111010101000011000101101001110011110011100100110010001100011100110111101011001100111010101100110010001100000010011101101111011110111011100101000010001011011100111101000010001011011100111111001101101110010011001111001111101000110000011100000111010000110001101101000011010101011110101111001111110011010011001000101101110011010100001000110001001110011100111101101001101000110001101111111011101010110011001000110001110011011100111100000111101000110010101111001111001100100011000100011011001010110011000101111111110011011010001100101011110011111001011111001111011010010011001100011011001010110011000101111111110011011010001100101011110011110011001100000111000001110100001100011011010000110011001000110001000110110011000000100111011011110111101110111001001100111100111100000111101000

In [683]:
seed(1)
reverse_network = initialize_network([8,18,18,8])


In [684]:
k

[[1, 1, 0, 1, 1, 1, 0, 1, [0, 0, 1, 1, 0, 0, 0, 0]],
 [1, 1, 0, 0, 1, 1, 1, 0, [0, 0, 1, 1, 0, 0, 0, 1]],
 [1, 1, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 0, 1, 0]],
 [1, 0, 1, 1, 1, 1, 1, 0, [0, 0, 1, 1, 0, 0, 1, 1]],
 [1, 0, 1, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 0, 0]],
 [0, 0, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 0, 1]],
 [1, 0, 0, 1, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 1, 0]],
 [1, 0, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 1, 1]],
 [1, 1, 0, 0, 1, 0, 1, 0, [0, 0, 1, 1, 1, 0, 0, 0]],
 [0, 0, 0, 1, 1, 1, 0, 0, [0, 0, 1, 1, 1, 0, 0, 1]],
 [1, 0, 1, 0, 1, 0, 1, 0, [0, 1, 1, 0, 0, 0, 0, 1]],
 [1, 0, 0, 1, 0, 1, 1, 1, [0, 1, 1, 0, 0, 0, 1, 0]],
 [1, 0, 0, 0, 1, 1, 0, 1, [0, 1, 1, 0, 0, 0, 1, 1]],
 [1, 0, 1, 1, 0, 1, 0, 0, [0, 1, 1, 0, 0, 1, 0, 0]],
 [1, 0, 0, 1, 1, 0, 0, 1, [0, 1, 1, 0, 0, 1, 0, 1]],
 [1, 0, 0, 0, 0, 0, 1, 1, [0, 1, 1, 0, 0, 1, 1, 0]],
 [1, 0, 0, 1, 1, 1, 0, 0, [0, 1, 1, 0, 0, 1, 1, 1]],
 [1, 1, 0, 1, 1, 1, 0, 0, [0, 1, 1, 0, 1, 0, 0, 0]],
 [1, 0, 1, 0, 0, 0, 0, 1, [0, 1, 1, 0, 1, 0, 0

In [685]:
n_inputs = len(k[0]) - 1
n_outputs = len(k[0][-1])
start = timeit.default_timer()
train_network(reverse_network, k, 0.5, 10000, n_outputs)
stop = timeit.default_timer()
print("time = {}".format(stop - start))

> epoch=99, lrate=0.500, error=223.557
> epoch=199, lrate=0.500, error=174.314
> epoch=299, lrate=0.500, error=158.024
> epoch=399, lrate=0.500, error=92.825
> epoch=499, lrate=0.500, error=46.072
> epoch=599, lrate=0.500, error=17.516
> epoch=699, lrate=0.500, error=13.383
> epoch=799, lrate=0.500, error=11.074
> epoch=899, lrate=0.500, error=9.811
> epoch=999, lrate=0.500, error=9.603
> epoch=1099, lrate=0.500, error=9.480
> epoch=1199, lrate=0.500, error=9.396
> epoch=1299, lrate=0.500, error=9.282
> epoch=1399, lrate=0.500, error=8.300
> epoch=1499, lrate=0.500, error=7.283
> epoch=1599, lrate=0.500, error=7.227
> epoch=1699, lrate=0.500, error=7.198
> epoch=1799, lrate=0.500, error=7.178
> epoch=1899, lrate=0.500, error=7.162
> epoch=1999, lrate=0.500, error=7.149
> epoch=2099, lrate=0.500, error=7.139
> epoch=2199, lrate=0.500, error=7.130
> epoch=2299, lrate=0.500, error=7.122
> epoch=2399, lrate=0.500, error=7.115
> epoch=2499, lrate=0.500, error=7.109
> epoch=2599, lrate=0.500

In [693]:
n = list()
for i in range(100):
    n.append(predict(reverse_network, k[i], 0))
    p = predict(reverse_network, k[i], 0)
    if p == k[i][-1]:
        print(p, end="---")
        print(k[i][-1])
    else:
        print(p, end="---")
        print(k[i][-1])
c = list()
for i in n:
    s = ""
    i = list(map(str,i))
    c.append(s.join(i))
len(list( dict.fromkeys(c)))

[1, 0, 0, 0, 1, 1, 1, 1]---[0, 0, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0]---[0, 0, 1, 1, 0, 0, 0, 1]
[1, 1, 1, 1, 1, 0, 0, 0]---[0, 0, 1, 1, 0, 0, 1, 0]
[0, 0, 0, 1, 1, 1, 1, 1]---[0, 0, 1, 1, 0, 0, 1, 1]
[1, 1, 0, 0, 1, 0, 1, 1]---[0, 0, 1, 1, 0, 1, 0, 0]
[1, 1, 1, 1, 1, 0, 1, 0]---[0, 0, 1, 1, 0, 1, 0, 1]
[1, 1, 1, 1, 1, 0, 1, 0]---[0, 0, 1, 1, 0, 1, 1, 0]
[1, 1, 1, 1, 1, 0, 1, 0]---[0, 0, 1, 1, 0, 1, 1, 1]
[0, 0, 0, 0, 1, 1, 1, 1]---[0, 0, 1, 1, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 1, 0]---[0, 0, 1, 1, 1, 0, 0, 1]
[1, 0, 1, 0, 1, 0, 0, 1]---[0, 1, 1, 0, 0, 0, 0, 1]
[1, 1, 1, 1, 0, 0, 1, 0]---[0, 1, 1, 0, 0, 0, 1, 0]
[1, 0, 0, 0, 1, 0, 1, 0]---[0, 1, 1, 0, 0, 0, 1, 1]
[1, 0, 1, 0, 0, 1, 1, 0]---[0, 1, 1, 0, 0, 1, 0, 0]
[0, 0, 0, 1, 1, 1, 0, 0]---[0, 1, 1, 0, 0, 1, 0, 1]
[1, 0, 0, 0, 1, 1, 0, 0]---[0, 1, 1, 0, 0, 1, 1, 0]
[1, 0, 0, 0, 1, 0, 1, 0]---[0, 1, 1, 0, 0, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 1, 0]---[0, 1, 1, 0, 1, 0, 0, 0]
[1, 1, 0, 1, 1, 0, 1, 0]---[0, 1, 1, 0, 1, 0, 0, 1]
[0, 0, 0, 0,

80

In [694]:
k

[[1, 1, 0, 1, 1, 1, 0, 1, [0, 0, 1, 1, 0, 0, 0, 0]],
 [1, 1, 0, 0, 1, 1, 1, 0, [0, 0, 1, 1, 0, 0, 0, 1]],
 [1, 1, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 0, 1, 0]],
 [1, 0, 1, 1, 1, 1, 1, 0, [0, 0, 1, 1, 0, 0, 1, 1]],
 [1, 0, 1, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 0, 0]],
 [0, 0, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 0, 1]],
 [1, 0, 0, 1, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 1, 0]],
 [1, 0, 0, 0, 0, 1, 1, 0, [0, 0, 1, 1, 0, 1, 1, 1]],
 [1, 1, 0, 0, 1, 0, 1, 0, [0, 0, 1, 1, 1, 0, 0, 0]],
 [0, 0, 0, 1, 1, 1, 0, 0, [0, 0, 1, 1, 1, 0, 0, 1]],
 [1, 0, 1, 0, 1, 0, 1, 0, [0, 1, 1, 0, 0, 0, 0, 1]],
 [1, 0, 0, 1, 0, 1, 1, 1, [0, 1, 1, 0, 0, 0, 1, 0]],
 [1, 0, 0, 0, 1, 1, 0, 1, [0, 1, 1, 0, 0, 0, 1, 1]],
 [1, 0, 1, 1, 0, 1, 0, 0, [0, 1, 1, 0, 0, 1, 0, 0]],
 [1, 0, 0, 1, 1, 0, 0, 1, [0, 1, 1, 0, 0, 1, 0, 1]],
 [1, 0, 0, 0, 0, 0, 1, 1, [0, 1, 1, 0, 0, 1, 1, 0]],
 [1, 0, 0, 1, 1, 1, 0, 0, [0, 1, 1, 0, 0, 1, 1, 1]],
 [1, 1, 0, 1, 1, 1, 0, 0, [0, 1, 1, 0, 1, 0, 0, 0]],
 [1, 0, 1, 0, 0, 0, 0, 1, [0, 1, 1, 0, 1, 0, 0